In [1]:
import re

StatementMeta(, 2c89596e-8409-4f41-aeec-123c8569318e, 3, Finished, Available, Finished)

In [2]:
# --- config ---
SILVER_TABLE = "silver.date_dimension"
GOLD_SCHEMA  = "gold"


StatementMeta(, 2c89596e-8409-4f41-aeec-123c8569318e, 4, Finished, Available, Finished)

In [3]:
# Each role can choose:
#   - name: table/view suffix
#   - start_date: lower bound (YYYY-MM-DD)
#   - offset_years: include rows where <offset_col> <= offset_years
#   - offset_col: one of "Offset_Year" (default) or "Offset_FiscalYear"

roles = [
    {"name": "quote_sent",   "start_date": "2015-01-01", "offset_years": 1, "offset_col": "Offset_Year"},
    {"name": "quote_closed", "start_date": "2015-01-01", "offset_years": 1, "offset_col": "Offset_Year"},
    {"name": "email_sent",   "start_date": "2015-01-01", "offset_years": 1, "offset_col": "Offset_Year"},
    {"name": "note_created", "start_date": "2015-01-01", "offset_years": 1, "offset_col": "Offset_Year"},
    {"name": "order",        "start_date": "2010-01-01", "offset_years": 1, "offset_col": "Offset_Year"},
    {"name": "master",        "start_date": "2010-01-01", "offset_years": 1, "offset_col": "Offset_Year"},
    # Example using fiscal offset instead of calendar:
    # {"name": "invoice", "start_date": "2017-07-01", "offset_years": 1, "offset_col": "Offset_FiscalYear"},
]

created = []
for r in roles:
    # validate + sanitise
    name = r["name"]
    start_date = r.get("start_date", "2015-01-01")
    offset_years = int(r.get("offset_years", 1))
    offset_col = r.get("offset_col", "Offset_Year")
    if offset_col not in ("Offset_Year", "Offset_FiscalYear"):
        raise ValueError(f"offset_col must be 'Offset_Year' or 'Offset_FiscalYear' (got {offset_col})")

    ident = re.sub(r"[^a-zA-Z0-9_]", "_", name.lower())
    temp_view = f"vw_date_{ident}_base"           # TEMP VIEW (session-scoped)
    table_name = f"{GOLD_SCHEMA}.dim_date_{ident}"# materialised Delta table

    # Create per-role TEMP VIEW
    spark.sql(f"""
        CREATE OR REPLACE TEMP VIEW {temp_view} AS
        SELECT *
        FROM {SILVER_TABLE}
        WHERE Date >= DATE '{start_date}'
          AND {offset_col} <= {offset_years}
    """)

    # Materialise as a TABLE (overwrite-safe CTAS)
    spark.sql(f"CREATE OR REPLACE TABLE {table_name} AS SELECT * FROM {temp_view}")

    created.append(table_name)

# Quick confirmation
print("Created/updated:", created)


StatementMeta(, 2c89596e-8409-4f41-aeec-123c8569318e, 5, Finished, Available, Finished)

Created/updated: ['gold.dim_date_quote_sent', 'gold.dim_date_quote_closed', 'gold.dim_date_email_sent', 'gold.dim_date_note_created', 'gold.dim_date_order', 'gold.dim_date_master']
